## Goroutines and Channels

Go enables two styles of concurrent programming. This chapter presents goroutines and channels, which support communicating sequential processes or CSP, a model of concurrency in which values are passed between independent activities (goroutines) but variables are for the most part confined to a single activity.

In Go, each concurrently executing activity is called a goroutine. Consider a program that has two functions, one that does some computation and one that writes some output, and assume that neither function calls the other. A sequential program may call one function and then call the other, but in a concurrent program with two or more goroutines, calls to both functions can be active at the same time.

In [1]:
// spinner
import (
    "fmt"
    "time"
)

func main() {
    go spinner(100 * time.Millisecond)
    const n = 35
    fibN := fib(n) // slow
    fmt.Printf("\rFibonacci(%d) = %d\n", n, fibN)
}

func spinner(delay time.Duration) {
    for {
        for _, r := range `-\|/` {
            fmt.Printf("\r%c", r)
            time.Sleep(delay)
        }
    }
}

func fib(x int) int {
    if x < 2 {
        return x
    }
    return fib(x-1) + fib(x-2)
}

main()

Fibonacci(35) = 9227465


## Channels
If goroutines are the activities of a concurrent Go program, channels are the connections between them.

Channels are a typed conduit through which you can send and receive values with the channel operator, <-

In [2]:
func sum(s []int, c chan int) {
    sum := 0
    for _, v := range s {
        sum += v
    }
    c <- sum // send sum to c
}

s := []int{7, 2, 8, -9, 4, 0}

c := make(chan int)
go sum(s[:len(s)/2], c)
go sum(s[len(s)/2:], c)
x, y := <-c, <-c // receive from c
fmt.Println(x, y, x + y)

-5 17 12


9 <nil>

A channel has two principal operations, send and receive, collectively known as communications. A send statement transmits a value from one goroutine, through the channel, to another goroutine executing a corresponding receive expression. Both operations are written using the <- operator.

In [3]:
func calc(i int, c chan int) {
    c <- 42 + i
}

In [4]:
nums := []int{1, 2, 3, 4, 5}
c := make(chan int)

// send them all for calculation
for i:= 0; i<len(nums); i++ {
    go calc(nums[i], c)
}

// get the results back
for i:= 0; i<len(nums); i++ {
    result := <- c
    fmt.Println(result)
}

47
43
44
45
46


Notice that you won't necessarily get results back in the order they were sent.

Channels can be buffered. Provide the buffer length as the second argument to make to initialize a buffered channel.

In [5]:
ch := make(chan int, 100)

In [6]:
type Square struct {
    Width int
}

func (s Square) Area() int {
    return s.Width * s.Width
}

In [7]:
s := Square{2}
s.Area()

4

In [8]:
squares := []Square{
    Square{1},
    Square{2},
    Square{3},
}

squares

[{1} {2} {3}]

In [9]:
areas := make(map[Square]int)

func recordArea(s Square) {
    areas[s] = s.Area()
}

for _, s := range squares {
    recordArea(s)
}

In [10]:
areas

map[{1}:1 {2}:4 {3}:9]

We can make this concurrent...

In [5]:
type Square struct {
    Width int
}

func (s Square) Area() int {
    return s.Width * s.Width
}

In [9]:
// see the results with conc.go
import "fmt"

func recordArea(s Square, areas map[Square]int, c chan bool) {
    areas[s] = s.Area()
    c <- true
}

func main() {
    squares := []Square{
        Square{1},
        Square{2},
        Square{3},
    }

    areas := make(map[Square]int)
    c := make(chan bool)

    for _, s := range squares {
        go recordArea(s, areas, c)
    }

    for i := 0; i < len(squares); i++ {
        <-c
    }

    fmt.Println(areas)
}


## Channel close

Closing a channel indicates that no more values will be sent on it. This can be useful to communicate completion to the channel’s receivers.

In [1]:
import "fmt"

jobs := make(chan int, 5)
done := make(chan bool)

go func() {
    for {
        j, more := <-jobs
        fmt.Println("... j, more -> ", j, more)
        if more {
            fmt.Println("received job", j)
        } else {
            fmt.Println("received all jobs")
            done <- true
            return
        }
    }
}()

for j := 1; j <= 3; j++ {
    jobs <- j
    fmt.Println("sent job", j)
}

close(jobs)
fmt.Println("sent all jobs")

<- done

sent job 1
sent job 2
sent job 3
sent all jobs
... j, more ->  1 true
received job 1
... j, more ->  2 true
received job 2
... j, more ->  3 true
received job 3
... j, more ->  0 false
received all jobs


true true

Communication over an unbuffered channel causes the sending and receiving goroutines to synchronize. Because of this, unbuffered channels are sometimes called synchronous channels. When a value is sent on an unbuffered channel, the receipt of the value happens before the reawakening of the sending goroutine.

## Pipelines
Channels can be used to connect goroutines together so that the output of one is the input to another. This is called a pipeline.

In [4]:
import "fmt"


func main() {
    naturals := make(chan int)
    squares := make(chan int)

    // Counter
    go func() {
        for x := 0; x<=10; x++ {
            naturals <- x
        }
        close(naturals)
    }()

    // Squarer
    go func() {
        for {
            x, more := <-naturals
            if more == false {
                break
            }
            squares <- x * x
        }
        close(squares)
    }()

    // Printer (in main goroutine)
    for {
        square, more := <- squares
        if more == false {
            break
        }
        fmt.Println(square)
    }
}

main()

0
1
4
9
16
25
36
49
64
81
100


## Unidirectional Channel Types

This arrangement is typical. When a channel is supplied as a function parameter, it is nearly always with the intent that it be used exclusively for sending or exclusively for receiving.

To document this intent and prevent misuse, the Go type system provides unidirectional channel types that expose only one or the other of the send and receive operations. The type chan<- int, a send-only channel of int, allows sends but not receives. Conversely, the type <-chan int, a receive-only channel of int, allows receives but not sends. (The position of the <- arrow relative to the chan keyword is a mnemonic.)

We can modify our previous program to reflect this...

In [8]:
import "fmt"


func counter(out chan<- int) {
    for x := 0; x <= 10; x++ {
        out <- x
    }
    close(out)
}

func squarer(out chan<- int, in <-chan int) {
    for v := range in {
        out <- v * v
    }
    close(out)
}

func printer(in <-chan int) {
    for v := range in {
        fmt.Println(v)
    }
}

func main() {
    naturals := make(chan int)
    squares := make(chan int)

    go counter(naturals)
    go squarer(squares, naturals)
    printer(squares)
}

main()

0
1
4
9
16
25
36
49
64
81
100


## Buffered Channels
In the unlikely event that a program needs to know the channel’s buffer capacity, it can be obtained by calling the built-in cap function:

In [9]:
c := make(chan int)
cap(c)

0

In [10]:
c := make(chan int, 10)
cap(c)

10

When applied to a channel, the built-in len function returns the number of elements currently buffered.

In [12]:
c <- 1
c <- 2
c <- 3

len(c)

3

In [13]:
cap(c)

10

In [14]:
for {
    if len(c) == 0 {
        break
    }
    fmt.Println(<-c)
}

1 true
2 true
3 true


In [20]:
func mirroredQuery() string {
    responses := make(chan string, 3)
    go func() { responses <- request("asia.gopl.io") }()
    go func() { responses <- request("europe.gopl.io") }()
    go func() { responses <- request("americas.gopl.io") }()
    return <-responses // return the quickest response
}

func request(hostname string) (response string) {
    return hostname + " returned"
}

mirroredQuery()

americas.gopl.io returned

## Multiplexing with select
The time.Tick function returns a channel on which it sends events periodically, acting like a metronome. The value of each event is a timestamp.

In [22]:
import (
    "fmt"
    "time"
)

func main() {
    fmt.Println("Commencing countdown.")
    tick := time.Tick(1 * time.Second)
    for countdown := 5; countdown > 0; countdown-- {
        fmt.Println(countdown)
        <-tick
    }
    launch()
}

func launch() {
    fmt.Println("Lift off!")
}

main()

Commencing countdown.
5
4
3
2
1
Lift off!


Now let’s add the ability to abort the launch sequence by pressing the return key during the countdown.

In [24]:
import (
    "fmt"
    "os"
    "time"
)

func main() {
    abort := make(chan struct{})
    go func() {
        os.Stdin.Read(make([]byte, 1)) // read a single byte
        abort <- struct{}{}
    }()


    fmt.Println("Commencing countdown.  Press return to abort.")
    select {
    case <-time.After(5 * time.Second):
        // Do nothing.
    case <-abort:
        fmt.Println("Launch aborted!")
        return
    }
    launch()
}

func launch() {
    fmt.Println("Lift off!")
}

main()

Commencing countdown.  Press return to abort.
Launch aborted!


Now each iteration of the countdown loop needs to wait for an event to arrive on one of the two channels: the ticker channel if everything is fine or an abort event if there was an "anomaly".

## Odds and evens

In [1]:
// go run oddevens.go

import "fmt"

func collectNumbers(numbers chan<- int) {
    for i := 1; i <= 10; i++ {
        numbers <- i
    }
    close(numbers)
}

func sortOddsAndEvens(numbers <-chan int, odd chan<- int, even chan<- int) {
    for {
        number, more := <-numbers
        if !more {
            break
        }
        if number%2 == 0 {
            even <- number
        } else {
            odd <- number
        }
    }
    close(odd)
    close(even)
}

func printNumbers(even <-chan int, odd <-chan int, done chan<- bool) {
    for {
        select {
        case n, more := <-even:
            if !more {
                even = nil
            } else {
                fmt.Println("even", n)
            }
        case n, more := <-odd:
            if !more {
                odd = nil
            } else {
                fmt.Println("odd", n)
            }
        }
        if even == nil && odd == nil {
            break
        }
    }
    done <- true
}

func main() {
    buff := 10
    numbers := make(chan int, buff)
    odd := make(chan int, buff)
    even := make(chan int, buff)
    done := make(chan bool)

    go collectNumbers(numbers)
    go sortOddsAndEvens(numbers, odd, even)
    go printNumbers(odd, even, done)

    <-done // wait
}